<a href="https://colab.research.google.com/github/OtienoSharon15/Alice_in_Wonderland/blob/main/Alice_wonderland_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INSTALLING LIBRARIES

In [1]:
%pip install langchain-groq langchain sentence-transformers langchain-community  pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.1 MB/s eta 

In [2]:
#importing libraries
from sentence_transformers import SentenceTransformer
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain_groq import ChatGroq


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()

Saving alice_in_wonderland.pdf to alice_in_wonderland.pdf


In [5]:
doc_path="/content/alice_in_wonderland.pdf"
loader= PyPDFLoader(doc_path)
docs=loader.load()
print(len(docs))

84


In [6]:
#initializing parent and text splitters
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100,
)

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
)


In [7]:
#initializing the embedding model
bge_model=SentenceTransformer("BAAI/bge-base-en-v1.5")
class BGEEmbeddings:
  def embed_documents(self, text):
    return bge_model.encode(text,batch_size=8,normalize_embeddings=True).tolist()
  def embed_query(self, query):
    return bge_model.encode([query],normalize_embeddings=True).tolist()[0]




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#creating the vector store
wonderland_vector_store=Chroma(
    collection_name="wonderland",
    embedding_function=BGEEmbeddings(),
    persist_directory="./chroma",
)



/tmp/ipython-input-783672564.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  wonderland_vector_store=Chroma(


In [9]:
#creating in-memory store
store=InMemoryStore()

In [10]:
#creating retrievers
retriever=ParentDocumentRetriever(
    vectorstore=wonderland_vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [11]:
#adding documents into the vector store
retriever.add_documents(docs)

In [13]:
from google.colab import userdata
api_key = userdata.get('alice_wonderland')

In [14]:
#creating a child retriever
child_retriever=wonderland_vector_store.as_retriever()

In [15]:
#initializing the LLM
llm=ChatGroq(
   groq_api_key=userdata.get('alice_wonderland'),
   model_name="llama-3.3-70b-versatile",
   temperature=0.7

)

In [16]:
#creating the template
template="""
 Answer the following questions according to the context given.
 If the question is out of context,just say the question is out of context Kindly give me another question.
 Do not try to answer a question out of context.
 {context}
 Question: {question}
"""


In [17]:
#creating the chain
qa_chain=PromptTemplate(template=template,input_variables=["context","question"])

In [18]:
chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":qa_chain},
    return_source_documents=True
)

In [19]:
question="How does the Queen of Hearts usually react when she is angry?"
answer=chain({"query":question})
print(answer["result"])

/tmp/ipython-input-1232832341.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer=chain({"query":question})


The Queen of Hearts usually reacts to anger by shouting "Off with their head!" and ordering executions. She is described as being "in a furious passion" and shouting this phrase "about once in a minute". This suggests that she has a quick temper and is prone to violent outbursts when she is angry.


In [20]:
question="who is Alice on wonderland"
answer=chain({"query":question})
print(answer["result"])

Alice is the main character in the story "Alice's Adventures in Wonderland". She is a young girl who falls down a rabbit hole and enters a fantastical world called Wonderland, where she encounters strange creatures and has many curious adventures.


In [21]:
question="what happens when Alice drinks from the bottle labelled drink me"
answer=chain({"query":question})
print(answer["result"])

When Alice drinks from the bottle labelled "DRINK ME", she finds the liquid very nice and decides to taste it. However, there is no description of what happens to her after drinking from this particular bottle in the provided context. 

But in another part of the context, it is mentioned that she drank from another bottle (not explicitly stated as the same one, but implied to be so) and found that it made her grow large very quickly, to the point where her head was pressing against the ceiling, and she had to kneel down and eventually lie down to avoid breaking her neck.
